In [ ]:
#  add langchain-experimental

In [1]:
from utils import get_vertex_ai_embedding, get_vertex_ai_model

In [2]:
text_embedding = get_vertex_ai_embedding()

c:\khajaclassroom\GenerativeAI\agenticai\aug25\kb_articles_rag\.venv\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [ ]:
# semantic chunker
from langchain_experimental.text_splitter import SemanticChunker
semantic_chunker = SemanticChunker(
    embeddings=text_embedding,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95
)

In [1]:
DOC = """
## 🔹 What is a Service Mesh?

A **Service Mesh** is a dedicated infrastructure layer that manages **service-to-service communication** in a microservices architecture. Instead of letting each microservice handle things like discovery, retries, security, and observability, a service mesh offloads these responsibilities to a dedicated set of components.

Think of it as a **"traffic controller"** for microservices, ensuring that communication between services is **secure, reliable, and observable**.

---

## 🔹 Key Features of a Service Mesh

1. **Traffic Management**

   * Intelligent routing (A/B testing, canary releases, blue/green deployments).
   * Load balancing between services.
   * Retry and timeout policies.

2. **Security**

   * Mutual TLS (mTLS) for encrypting service-to-service communication.
   * Authentication and authorization between services.
   * Identity management.

3. **Observability**

   * Distributed tracing (knowing how requests flow through services).
   * Metrics (latency, error rates, throughput).
   * Logging of service interactions.

4. **Reliability**

   * Circuit breaking (stop sending requests to failing services).
   * Rate limiting (avoid overloading services).
   * Failover handling.

---

## 🔹 How it Works

A service mesh typically uses a **sidecar proxy** pattern:

* Each microservice instance runs alongside a **proxy** (usually Envoy).
* All traffic in and out of the service goes through the proxy.
* The proxies communicate with a **control plane**, which manages policies and configurations.

So, developers write business logic, and the service mesh takes care of **networking concerns**.

---

## 🔹 Popular Service Mesh Implementations

| Service Mesh                | Key Highlights                                                |
| --------------------------- | ------------------------------------------------------------- |
| **Istio**                   | Most popular, feature-rich, uses Envoy as sidecar.            |
| **Linkerd**                 | Lightweight, simple, built for Kubernetes.                    |
| **Consul Connect**          | From HashiCorp, integrates with Consul for service discovery. |
| **Kuma**                    | CNCF project by Kong, easy to use, multi-cluster support.     |
| **Open Service Mesh (OSM)** | Lightweight, from Microsoft, based on Envoy.                  |

---

## 🔹 When to Use a Service Mesh?

* When you have **many microservices** and need to manage complex communication.
* If you need **zero-trust security** with mTLS between services.
* For **advanced deployment strategies** (canary, A/B testing).
* When you require **deep observability** into service interactions.

---

✅ **In simple terms**:
A Service Mesh helps you **secure, monitor, and control** communication between microservices, without changing your application code.
"""

In [5]:
from utils import get_fixed_size_chunker, get_recursive_chunker, get_semantic_chunker, get_vertex_ai_embedding

In [6]:
fixed = get_fixed_size_chunker()
recursive = get_recursive_chunker()
text_embedding = get_vertex_ai_embedding()
semantic = get_semantic_chunker(embedding=text_embedding)

c:\khajaclassroom\GenerativeAI\agenticai\aug25\kb_articles_rag\.venv\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [7]:
fixed_chunks = fixed.split_text(DOC)
recursive_chunks = recursive.split_text(DOC)
semantic_chunks = semantic.split_text(DOC)

In [8]:
print(f"Counts \nfixed: {len(fixed_chunks)} \nrecursive: {len(recursive_chunks)} \nsemantic: {len(semantic_chunks)}")

Counts 
fixed: 6 
recursive: 7 
semantic: 3


In [9]:
# Build three vector stores
from utils import vector_store_from_chunks


In [10]:
vs_fixed = vector_store_from_chunks(
    chunks=fixed_chunks,
    embedding=text_embedding,
    name="fixed_demo",
    persist_directory="./db/fixed/vectordb"
)

In [11]:
vs_recursive = vector_store_from_chunks(
    chunks=recursive_chunks,
    embedding=text_embedding,
    name="recursive_demo",
    persist_directory="./db/recursive/vectordb"
)

In [12]:
vs_semantic = vector_store_from_chunks(
    chunks=semantic_chunks,
    embedding=text_embedding,
    name="semantic_demo",
    persist_directory="./db/semantic/vectordb"
)

In [26]:
retriver_fixed = vs_fixed.as_retriever(
    search_kwargs={"k": 1}
)
retriver_recursive = vs_recursive.as_retriever(
    search_kwargs={"k": 1}
)
retriver_semantic = vs_semantic.as_retriever(
    search_kwargs={"k": 1}
)

In [27]:
QUERIES = [
    "How do i enable mutual TLS between services"
]

In [28]:
def display_response(response: list):
    for doc in response:
        print(doc.page_content)

In [29]:
fixed_response = retriver_fixed.invoke(QUERIES[0])

In [30]:
display_response(fixed_response)

---
## 🔹 When to Use a Service Mesh?
* When you have **many microservices** and need to manage complex communication.
* If you need **zero-trust security** with mTLS between services.
* For **advanced deployment strategies** (canary, A/B testing).
* When you require **deep observability** into service interactions.
---
✅ **In simple terms**:
A Service Mesh helps you **secure, monitor, and control** communication between microservices, without changing your application code.


In [31]:
recursive_response = retriver_recursive.invoke(QUERIES[0])

In [32]:
display_response(recursive_response)

Think of it as a **"traffic controller"** for microservices, ensuring that communication between services is **secure, reliable, and observable**.

---

## 🔹 Key Features of a Service Mesh

1. **Traffic Management**

   * Intelligent routing (A/B testing, canary releases, blue/green deployments).
   * Load balancing between services.
   * Retry and timeout policies.

2. **Security**

   * Mutual TLS (mTLS) for encrypting service-to-service communication.


In [33]:
semantic_response = retriver_semantic.invoke(QUERIES[0])

In [34]:
display_response(semantic_response)

|
| **Consul Connect**          | From HashiCorp, integrates with Consul for service discovery. |
| **Kuma**                    | CNCF project by Kong, easy to use, multi-cluster support. |
| **Open Service Mesh (OSM)** | Lightweight, from Microsoft, based on Envoy. |

---

## 🔹 When to Use a Service Mesh? * When you have **many microservices** and need to manage complex communication. * If you need **zero-trust security** with mTLS between services. * For **advanced deployment strategies** (canary, A/B testing). * When you require **deep observability** into service interactions. ---

✅ **In simple terms**:
A Service Mesh helps you **secure, monitor, and control** communication between microservices, without changing your application code. 
